# mTSP Environment

Differences with CVRP:

- We have a fixed maximum number of vehicles
- No capacity constraint
- Objective: minimize maximum subtour length (minmax)

In [5]:
import sys; sys.path.append(2*"../")

from typing import Optional

import numpy as np
import torch
from tensordict.tensordict import TensorDict

from torchrl.data import (
    BoundedTensorSpec,
    CompositeSpec,
    UnboundedContinuousTensorSpec,
    UnboundedDiscreteTensorSpec,
)

from rl4co.envs.utils import batch_to_scalar
from rl4co.envs.base import RL4COEnvBase


In [58]:
def _reset(self, td: Optional[TensorDict] = None, batch_size=None) -> TensorDict:
    # Initialize data
    if batch_size is None:
        batch_size = self.batch_size if td is None else td["observation"].shape[:-2]

    device = td.device if td is not None else self.device
    if td is None or td.is_empty():
        td = self.generate_data(batch_size=batch_size)

    # Keep track of the agent number to know when to stop
    agent_idx = torch.zeros((*batch_size, 1), dtype=torch.int64, device=device)
    
    # Make variable for max_subtour_length between subtours
    max_subtour_length = torch.zeros(batch_size, dtype=torch.float32, device=device)
    current_length = torch.zeros(batch_size, dtype=torch.float32, device=device)

    # Other variables
    current_node = torch.zeros((*batch_size, 1), dtype=torch.int64, device=device)
    available = torch.ones(
        (*batch_size, self.num_loc), dtype=torch.bool, device=device
    )  # 1 means not visited, i.e. action is allowed
    available[..., 0] = 0  # Depot is not available as first node
    i = torch.zeros((*batch_size, 1), dtype=torch.int64, device=device)
    
    return TensorDict(
        {
            "observation": td['observation'], # depot is first node
            "num_agents": td['num_agents'],
            "max_subtour_length": max_subtour_length,
            "current_length": current_length,
            "agent_idx": agent_idx,
            "first_node": current_node,
            "current_node": current_node,
            "i": i,
            "action_mask": available,
        },
        batch_size=batch_size,
    )

In [60]:
def _step(td: TensorDict) -> TensorDict:
    is_first_action = batch_to_scalar(td["i"]) == 0
    current_node = td["action"]
    first_node = current_node if is_first_action else td["first_node"]
    locs = td["observation"]

    # If current_node is the depot, then increment agent_idx
    cur_agent_idx = td["agent_idx"] + (current_node == 0).long()

    # Get cur distance: distance between current_node and td['current_node'] (which is the previous)
    cur_dist = torch.norm(locs.gather(-2, current_node.unsqueeze(-1).expand_as(locs)) - locs.gather(-2, td["current_node"].unsqueeze(-1).expand_as(locs)), p=2, dim=-1)

    # If current agent is different from previous agent, then we have a new subtour
    # We update the max_subtour_length and reset the current_length
    max_subtour_length = torch.where(td["current_length"] > td["max_subtour_length"], td["current_length"], td["max_subtour_length"])

    # If current agent is different from previous agent, then we have a new subtour, otherwise we add the distance
    current_length = torch.where(cur_agent_idx != td["agent_idx"], torch.zeros_like(td["current_length"]), td["current_length"] + cur_dist)


    # Set not visited to 0 (i.e., we visited the node)
    available = td["action_mask"].scatter(
        -1, current_node.unsqueeze(-1).expand_as(td["action_mask"]), 0
    )
    # Available[..., 0] is the depot, which is always available unless:
    # - current_node is the depot
    # - agent_idx greater than num_agents -1
    available[..., 0] = torch.logical_and(current_node != 0, td['agent_idx'] < td['num_agents'] - 1)

    # We are done there are no unvisited locations except the depot
    done = torch.count_nonzero(available[..., 1:], dim=-1) == 0

    # If done is True, then we make the depot available again, so that it will be selected as the next node with prob 1 (since we finished)
    available[..., 0] = done # where done = 1, available = 1

    # The reward is the negative of the max_subtour_length (minmax objective)
    reward = - max_subtour_length

    # The output must be written in a ``"next"`` entry
    return TensorDict(
        {
            "next": {
                "observation": td["observation"],
                "num_agents": td['num_agents'],
                "max_subtour_length": max_subtour_length,
                "current_length": current_length,
                "agent_idx": cur_agent_idx,
                "first_node": first_node,
                "current_node": current_node,
                "i": td["i"] + 1,
                "action_mask": available,
                "reward": reward,
                "done": done,
            }
        },
        td.shape,
    )

In [61]:
def _make_spec(self, td_params: TensorDict):
    """Make the observation and action specs from the parameters."""
    self.observation_spec = CompositeSpec(
        observation=BoundedTensorSpec(
            minimum=self.min_loc,
            maximum=self.max_loc,
            shape=(self.num_loc, 2),
            dtype=torch.float32,
        ),
        first_node=UnboundedDiscreteTensorSpec(
            shape=(1),
            dtype=torch.int64,
        ),
        current_node=UnboundedDiscreteTensorSpec(
            shape=(1),
            dtype=torch.int64,
        ),
        num_agents=UnboundedDiscreteTensorSpec(
            shape=(1),
            dtype=torch.int64,
        ),
        agent_idx=UnboundedDiscreteTensorSpec(
            shape=(1),
            dtype=torch.int64,
        ),
        i=UnboundedDiscreteTensorSpec(
            shape=(1),
            dtype=torch.int64,
        ),
        action_mask=UnboundedDiscreteTensorSpec(
            shape=(self.num_loc),
            dtype=torch.bool,
        ),
        shape=(),
    )
    self.input_spec = self.observation_spec.clone()
    self.action_spec = BoundedTensorSpec(
        shape=(1,),
        dtype=torch.int64,
        minimum=0,
        maximum=self.num_loc,
    )
    self.reward_spec = UnboundedContinuousTensorSpec(shape=(1,))
    self.done_spec = UnboundedDiscreteTensorSpec(shape=(1,), dtype=torch.bool)

In [62]:
def generate_data(self, batch_size) -> TensorDict:

    # Batch size input check
    batch_size = [batch_size] if isinstance(batch_size, int) else batch_size

    # Initialize the locations (including the depot which is always the first node)
    locs = (
        torch.FloatTensor(*batch_size, self.num_loc, 2)
        .uniform_(self.min_loc, self.max_loc)
        .to(self.device)
    )
    
    # Initialize the num_agents
    num_agents = torch.randint(
        self.min_num_agents, self.max_num_agents, size=batch_size, device=self.device
    )

    return TensorDict(
        {
            "observation": locs,
            "num_agents": num_agents,
        },
        batch_size=batch_size,
    )

In [64]:
def get_reward(self, td, actions) -> TensorDict:

    if self.cost_type == "minmax":
        # With minmax, get the maximum distance among subtours, calculated in the model
        return td["reward"]
    
    elif self.cost_type == "distance":
        locs = td["observation"]
        # Gather locations in order of tour
        locs = locs.gather(1, actions.unsqueeze(-1).expand_as(locs))

        # Concatenate as last node the depot
        locs = torch.cat([locs, locs[..., 0:1]], dim=-2)
        locs_next = torch.roll(locs, 1, dims=1)
        return  -((locs_next - locs).norm(p=2, dim=2).sum(1))
    else:
        raise ValueError(f"Cost type {self.cost_type} not supported")

In [65]:
class MTSPEnv(RL4COEnvBase):
    name = "mtsp"

    def __init__(
        self,
        num_loc: int = 10,
        min_loc: float = 0,
        max_loc: float = 1,
        min_num_agents: int = 1,
        max_num_agents: int = 10,
        cost_type: str = "minmax",
        td_params: TensorDict = None,
        seed: int = None,
        device: str = "cpu",
    ):
        self.num_loc = num_loc
        self.min_loc = min_loc
        self.max_loc = max_loc
        self.min_num_agents = min_num_agents
        self.max_num_agents = max_num_agents
        self.cost_type = cost_type

        super().__init__(device=device, batch_size=[])
        self._make_spec(td_params)
        if seed is None:
            seed = torch.empty((), dtype=torch.int64).random_().item()
        self.set_seed(seed)

    # Helpers: _make_step and gen_params
    # gen_params = gen_params # we don't use it for this case. See notebook 0
    _make_spec = _make_spec

    # Mandatory methods: _step, _reset and _set_seed
    _reset = _reset
    _step = _step

    # Get reward
    get_reward = get_reward

    # Data
    generate_data = generate_data

In [66]:
env = MTSPEnv()

td = env.generate_data(2)
print(td['observation'])
print(td['num_agents'])

tensor([[[0.1807, 0.7539],
         [0.2895, 0.9399],
         [0.6412, 0.0321],
         [0.7584, 0.1472],
         [0.4810, 0.3595],
         [0.2817, 0.7075],
         [0.2506, 0.0151],
         [0.1024, 0.2672],
         [0.4661, 0.3446],
         [0.4184, 0.2753]],

        [[0.9895, 0.6966],
         [0.3829, 0.7444],
         [0.2576, 0.9897],
         [0.3450, 0.6744],
         [0.5010, 0.5251],
         [0.2894, 0.2769],
         [0.2377, 0.8089],
         [0.9846, 0.4740],
         [0.1922, 0.4611],
         [0.0252, 0.5431]]])
tensor([8, 8])
